#### Dependency + imports

In [1]:
%%capture
%pip install pyyaml pandas;
%pip install -e .;
#to do fetch from git url

In [8]:
#imports
import pandas as pd
import subprocess
import os
from IPython.display import display, HTML
from tabulate import tabulate
from summary.report_fhir_counts import AVAILABLE_STUDIES
import yaml
import filecmp
from style import css

#### Generate summary files

In [3]:
%%capture
%run run-summaries.py

### Check if QA and Prod are synced

In [4]:
#run diff on QA and PROD
def folder_diff(folder1, folder2):
    """Compares two folders and generates HTML output for the differences or "All good!" if identical."""
    result = filecmp.dircmp(folder1, folder2)

    # Check if folders are identical
    if not result.left_only and not result.right_only and not result.diff_files:
        return f"<h3>Comparing '{folder1}' and '{folder2}'</h3><h2>✅ All good!</h2>" 

    # If not identical, generate the diff output
    diff_output = f"<h1>‼️ 🔴 🆘</h1><h3>'{folder1}' and '{folder2}' are not synced!</h3><ul>"

    if result.left_only:
        diff_output += "<li><b>Files only in {folder1}:</b><ul>"
        for item in result.left_only:
            diff_output += f"<li>{item}</li>"
        diff_output += "</ul></li>"

    if result.right_only:
        diff_output += "<li><b>Files only in {folder2}:</b><ul>"
        for item in result.right_only:
            diff_output += f"<li>{item}</li>"
        diff_output += "</ul></li>"

    if result.diff_files:
        diff_output += "<li><b>Different files:</b><ul>"
        for item in result.diff_files:
            diff_output += f"<li>{item}</li>"
        diff_output += "</ul></li>"

    # Recursively compare subdirectories
    for common_dir in result.common_dirs:
        diff_output += folder_diff(os.path.join(folder1, common_dir), os.path.join(folder2, common_dir))

    diff_output += "</ul>"
    return diff_output


# Replace with the actual paths to your folders
PROD = 'output/summary/PROD'
QA = 'output/summary/QA'

# Display the diff output as HTML
HTML("<div style='padding:1rem; border:1px solid; background:#eeee;'>" + folder_diff(PROD, QA) + "</div>")

<IPython.core.display.HTML object>

### Summaries

In [10]:
#display summaries
for study in AVAILABLE_STUDIES:
    with open(f"output/summary/PROD/fhir_summary_{study}.yaml", 'r') as file:
        yaml_content = yaml.safe_load(file)
    cards_html = ""
    for section, values in yaml_content.items():
        card_content = f"<h3>{section}</h3><ul>"
        if isinstance(values, dict):  # Check if 'values' is a dictionary
            for key, value in values.items():
                card_content += f"<li>{key}: {value}</li>"
        else:
            card_content += f"<li>{values}</li>"  # Handle single values
        card_content += "</ul>"
        cards_html += f'<div class="card">{card_content}</div>'

        style = f"<style>{css}</style>"
    display(HTML(style + f"<h2>{study}</h2>" +"<div class='container'>" + cards_html +"</div>"))


<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>

<IPython.core.display.HTML object>